In [50]:
import os
import logging
from typing import List, Optional, TypedDict, Dict, Any, Callable, Tuple
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import pandas as pd
import openai
import random
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain.schema import BaseMessage
import langgraph.graph as lg
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain_core.documents import Document
from langgraph.graph import StateGraph, START, END

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 후보 문서 정의
class CandidateDocument(TypedDict):
    name: str
    summary: str
    # 추가 필드들

# 평가 점수 정의
class EvaluationScores(TypedDict):
    team_trustworthiness: int
    customer_feedback: int
    risk_analysis: int
    founder_commitment: int

# 그래프 상태 정의
class GraphState(TypedDict):
    messages: List
    user_query: str
    candidates_documents: List[CandidateDocument]
    domain: Optional[str]
    evaluation_scores: Optional[EvaluationScores]
    evaluation_summary: Optional[str]

# 글로벌 변수로 데이터/모델 초기화
startup_data = {}  # CSV 데이터를 저장할 딕셔너리
embedding_model = None  # 임베딩 모델
pinecone_index = None  # Pinecone 인덱스
llm = None  # LLM 모델

# 초기화 함수
def init_resources(pinecone_api_key: str, openai_api_key: str, index_name: str, csv_path: str = None):
    """필요한 리소스 초기화"""
    global startup_data, embedding_model, pinecone_index, llm
    
    # OpenAI 설정
    openai.api_key = openai_api_key
    llm = ChatOpenAI(api_key=openai_api_key, model="gpt-3.5-turbo", temperature=0.7)
    
    # 임베딩 모델 로드
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
    
    # Pinecone 클라이언트 초기화
    pc = Pinecone(api_key=pinecone_api_key)
    pinecone_index = pc.Index(index_name)
    
    # CSV 데이터 로드 (있는 경우)
    if csv_path:
        try:
            df = pd.read_csv(csv_path)
            # 스타트업 이름을 키로 하고 관련 텍스트를 값으로 하는 딕셔너리 생성
            for _, row in df.iterrows():
                startup_name = row['startup']
                if startup_name not in startup_data:
                    startup_data[startup_name] = []
                startup_data[startup_name].append(row['text'])
            logger.info(f"CSV 파일에서 {len(startup_data)} 개의 스타트업 데이터를 로드했습니다.")
        except Exception as e:
            logger.error(f"CSV 파일 로드 중 오류 발생: {e}")
    
    logger.info(f"리소스 초기화 완료. 인덱스: {index_name}")

# 텍스트 데이터 요약 함수
def summarize_startup_data(startup_name: str, texts: List[str], max_texts: int = 50) -> str:
    """스타트업 관련 텍스트 데이터를 요약하여 반환"""
    # 텍스트가 너무 많으면 일부만 선택 (토큰 제한 회피)
    if len(texts) > max_texts:
        # 앞에서 일부만 선택
        selected_texts = texts[:max_texts]
        combined_text = f"{startup_name}에 관한 정보 (총 {len(texts)}개 중 {max_texts}개 샘플): \n\n" + "\n".join(selected_texts)
    else:
        combined_text = f"{startup_name}에 관한 정보: \n\n" + "\n".join(texts)
    
    return combined_text

# LangGraph 노드 함수들
def process_user_query(state: GraphState) -> GraphState:
    """사용자 쿼리 처리"""
    # 최신 메시지 가져오기
    user_message = state["messages"][-1]["content"]
    
    # 상태 업데이트
    state["user_query"] = user_message
    
    return state

def search_startups(state: GraphState) -> GraphState:
    """Pinecone에서 유사한 스타트업 검색 - 가장 유사한 1개만 반환"""
    query = state["user_query"]
    
    try:
        # 쿼리 임베딩 생성
        query_embedding = embedding_model.encode(query).tolist()
        
        # Pinecone에서 가장 유사한 1개의 벡터만 검색
        results = pinecone_index.query(
            vector=query_embedding,
            top_k=1,  # 가장 유사한 1개만 가져오기
            include_metadata=True
        )
        
        # 결과 처리 - 스타트업 이름만 가져오기
        candidates: List[CandidateDocument] = []
        for match in results.matches:
            if match.metadata and "name" in match.metadata:
                # CandidateDocument 형식에 맞게 데이터 추가
                candidate: CandidateDocument = {
                    "name": match.metadata.get("name", "Unknown"),
                    "summary": ""  # 초기 빈 문자열, 나중에 LLM으로 채울 예정
                }
                candidates.append(candidate)
        
        # 결과 저장
        state["candidates_documents"] = candidates
        
        # 도메인 추출 (스타트업 텍스트 데이터 기반으로 수정)
        if candidates:
            startup_name = candidates[0]["name"]
            startup_texts = startup_data.get(startup_name, [])
            
            if startup_texts:
                # OpenAI API를 사용하여 텍스트에서 도메인 추출
                combined_text = "\n".join(startup_texts[:10])  # 처음 10개 텍스트만 사용
                
                try:
                    prompt = f"""
다음은 '{startup_name}'라는 스타트업에 관한 텍스트 데이터입니다:

{combined_text}

위 정보를 바탕으로, 이 스타트업이 속한 도메인을 아래 목록 중에서 하나만 선택하여 답변해주세요:
헬스케어, AI, 핀테크, 에듀테크, 푸드테크, 커머스, 교육, 모빌리티, 에너지, 환경, 소프트웨어, 하드웨어, 소셜미디어, 엔터테인먼트, 부동산, 보안

한 단어로만 대답해주세요. 위 목록에 없다면 가장 근접한 것을 선택하거나 '기타'라고 답변하세요.
"""
                    response = openai.chat.completions.create(
                        model="gpt-3.5-turbo",
                        messages=[
                            {"role": "system", "content": "당신은 스타트업 분석가입니다. 주어진 정보를 바탕으로 스타트업의 도메인을 분류하세요."},
                            {"role": "user", "content": prompt}
                        ],
                        max_tokens=10,
                        temperature=0.3
                    )
                    
                    domain = response.choices[0].message.content.strip()
                    state["domain"] = domain
                    logger.info(f"스타트업 '{startup_name}'의 도메인으로 '{domain}'을(를) 추출했습니다.")
                    
                except Exception as e:
                    logger.error(f"도메인 추출 중 OpenAI API 오류: {e}")
                    state["domain"] = "기타"  # 기본값
            else:
                state["domain"] = "기타"
        else:
            state["domain"] = None
        
        if candidates:
            logger.info(f"쿼리와 가장 유사한 스타트업을 찾았습니다: {candidates[0]['name']}")
        else:
            logger.info("관련 스타트업을 찾지 못했습니다.")
        
    except Exception as e:
        logger.error(f"유사 스타트업 검색 중 오류 발생: {e}")
        state["candidates_documents"] = []
    
    return state

def generate_summaries(state: GraphState) -> GraphState:
    """선택된 스타트업에 대한 설명 생성 - 토큰 제한 고려"""
    candidates = state["candidates_documents"]
    
    if not candidates:
        return state
    
    # 가장 유사한 하나의 스타트업만 처리
    candidate = candidates[0]
    startup_name = candidate["name"]
    startup_info = startup_data.get(startup_name, [])
    
    if startup_info:
        try:
            # 텍스트 데이터 요약 (토큰 제한 고려)
            summarized_info = summarize_startup_data(startup_name, startup_info)
            
            # 프롬프트 생성 - 더 간결하게
            prompt = f"""
스타트업 '{startup_name}'에 대한 일부 정보:

{summarized_info}

위 정보를 바탕으로 이 스타트업이 어떤 사업을 하는지 2-3문장으로 간결하게 설명해주세요.
정보가 제한적이거나 불확실하다면, '정보가 제한적이지만, 추측하자면...'이라고 명시해주세요.
"""
            # OpenAI API 호출
            response = openai.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "당신은 스타트업 분석가입니다. 주어진 정보를 바탕으로 간결한 설명을 작성하세요."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=150,
                temperature=0.7
            )
            
            # 응답에서 설명 추출
            summary = response.choices[0].message.content.strip()
            candidate["summary"] = summary
            logger.info(f"스타트업 설명 생성 완료: {startup_name}")
            
        except Exception as e:
            logger.error(f"LLM으로 설명 생성 중 오류 발생: {e}")
            # 오류 상황에서 간단한 대체 문구 제공
            candidate["summary"] = f"{startup_name}은(는) 데이터 기반 추정: 텍스트 정보 분석 중 문제가 발생했습니다."
    else:
        candidate["summary"] = "이 스타트업에 대한 상세 정보가 없습니다."
    
    return state

def generate_evaluation_scores(state: GraphState) -> GraphState:
    """스타트업에 대한 평가 점수 생성"""
    if not state["candidates_documents"]:
        state["evaluation_scores"] = None
        return state
    
    try:
        candidate = state["candidates_documents"][0]
        startup_name = candidate["name"]
        startup_summary = candidate["summary"]
        
        # 임의의 점수를 생성하는 대신 OpenAI를 사용해 평가 점수 생성
        prompt = f"""
스타트업 '{startup_name}'에 대한 설명:

{startup_summary}

위 정보를 바탕으로 다음 4가지 측면에서 1-10점 사이의 점수를 제시해주세요(숫자만):
1. 팀 신뢰성(team_trustworthiness): 창업자와 팀은 이 분야에서 얼마나 믿을만한가?
2. 고객 피드백(customer_feedback): 초기 고객의 반응은 어떠한가?
3. 리스크 분석(risk_analysis): 기술, 운영, 법률적 리스크는 어떠한가? (점수가 높을수록 리스크가 낮음)
4. 창업자 헌신도(founder_commitment): 창업자가 다음 10년을 이 분야에 쏟아부을 각오가 있는가?

답변 형식: 팀 신뢰성: X, 고객 피드백: Y, 리스크 분석: Z, 창업자 헌신도: W
"""
        
        # OpenAI API 호출
        try:
            response = openai.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "당신은 스타트업 투자 평가 전문가입니다. 주어진 정보를 바탕으로 객관적인 평가 점수를 제시하세요."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=100,
                temperature=0.7
            )
            
            # 응답 처리
            response_text = response.choices[0].message.content.strip()
            logger.info(f"평가 점수 응답: {response_text}")
            
            # 점수 추출 시도
            try:
                # 응답 파싱 시도
                scores = {}
                
                # 각 점수 항목 추출
                if "팀 신뢰성" in response_text:
                    team_trust_text = response_text.split("팀 신뢰성")[1].split(",")[0]
                    team_trust = int(''.join(filter(str.isdigit, team_trust_text)))
                    scores["team_trustworthiness"] = team_trust
                else:
                    scores["team_trustworthiness"] = 7  # 기본값
                
                if "고객 피드백" in response_text:
                    customer_feedback_text = response_text.split("고객 피드백")[1].split(",")[0]
                    customer_feedback = int(''.join(filter(str.isdigit, customer_feedback_text)))
                    scores["customer_feedback"] = customer_feedback
                else:
                    scores["customer_feedback"] = 6  # 기본값
                
                if "리스크 분석" in response_text:
                    risk_analysis_text = response_text.split("리스크 분석")[1].split(",")[0]
                    risk_analysis = int(''.join(filter(str.isdigit, risk_analysis_text)))
                    scores["risk_analysis"] = risk_analysis
                else:
                    scores["risk_analysis"] = 5  # 기본값
                
                if "창업자 헌신도" in response_text:
                    founder_commitment_text = response_text.split("창업자 헌신도")[1].split(",")[0]
                    founder_commitment = int(''.join(filter(str.isdigit, founder_commitment_text)))
                    scores["founder_commitment"] = founder_commitment
                else:
                    scores["founder_commitment"] = 8  # 기본값
                
                # 점수가 1-10 사이인지 확인
                for key, value in scores.items():
                    if not (1 <= value <= 10):
                        scores[key] = min(max(value, 1), 10)  # 1-10 사이로 제한
                
                # 평가 점수 저장
                state["evaluation_scores"] = scores
                logger.info(f"생성된 평가 점수: {scores}")
                
            except Exception as parsing_error:
                logger.error(f"평가 점수 파싱 오류: {parsing_error}")
                # 파싱 실패 시 기본 점수 사용
                state["evaluation_scores"] = {
                    "team_trustworthiness": 7,
                    "customer_feedback": 6,
                    "risk_analysis": 5,
                    "founder_commitment": 8
                }
                
        except Exception as api_error:
            logger.error(f"OpenAI API 호출 오류: {api_error}")
            # API 호출 실패 시 기본 점수 사용
            state["evaluation_scores"] = {
                "team_trustworthiness": 7,
                "customer_feedback": 6,
                "risk_analysis": 5,
                "founder_commitment": 8
            }
    
    except Exception as e:
        logger.error(f"평가 점수 생성 중 오류 발생: {e}")
        # 기본 점수 설정
        state["evaluation_scores"] = {
            "team_trustworthiness": 7,
            "customer_feedback": 6,
            "risk_analysis": 5,
            "founder_commitment": 8
        }
    
    return state

def generate_evaluation_summary(state: GraphState) -> GraphState:
    """OpenAI를 사용하여 평가 요약 생성"""
    try:
        if not state["candidates_documents"] or not state["evaluation_scores"]:
            state["evaluation_summary"] = None
            return state
        
        # 첫 번째 후보 문서와 평가 점수 사용
        candidate = state["candidates_documents"][0]
        scores = state["evaluation_scores"]
        domain = state["domain"] or "기술"
        
        # 프롬프트 구성
        prompt = f"""
다음 {domain} 스타트업에 대한 평가 요약을 작성해주세요:

스타트업 이름: {candidate['name']}
요약: {candidate['summary']}
도메인: {domain}

평가 점수:
- 팀 신뢰성: {scores['team_trustworthiness']}/10
- 고객 피드백: {scores['customer_feedback']}/10
- 리스크 분석: {scores['risk_analysis']}/10
- 창업자 헌신도: {scores['founder_commitment']}/10

평가 요약은 각 점수에 대한 설명과 함께 종합적인 의견을 포함해야 합니다.
투자자의 관점에서 이 스타트업의 잠재력과 위험 요소를 균형 있게 평가해주세요.
요약은 200자 이내로 작성해주세요.
"""
        
        # OpenAI API 호출
        try:
            response = openai.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "당신은 스타트업 투자 평가 전문가입니다. 투자자 관점에서 객관적이고 유용한 평가 요약을 작성하세요."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=250,
                temperature=0.7
            )
            
            # 응답에서 요약 추출
            summary = response.choices[0].message.content.strip()
            state["evaluation_summary"] = summary
            logger.info(f"생성된 평가 요약: {summary[:100]}...")
            
        except Exception as api_error:
            logger.error(f"평가 요약 생성 API 호출 오류: {api_error}")
            # 기본 요약 생성
            state["evaluation_summary"] = f"{candidate['name']}은(는) {domain} 분야의 스타트업으로, 팀 신뢰성({scores['team_trustworthiness']}점)과 창업자 헌신도({scores['founder_commitment']}점)는 높은 편이나, 고객 피드백({scores['customer_feedback']}점)과 리스크 분석({scores['risk_analysis']}점) 측면에서 개선이 필요합니다. 종합적으로 잠재력이 있으나 주의 깊은 관찰이 필요한 투자 대상입니다."
        
    except Exception as e:
        logger.error(f"평가 요약 생성 중 오류 발생: {e}")
        state["evaluation_summary"] = "평가 요약을 생성할 수 없습니다."
    
    return state

def prepare_response(state: GraphState) -> GraphState:
    """응답 메시지 생성 및 상태에 추가"""
    if state["candidates_documents"]:
        # 가장 유사한 스타트업 정보만 포함
        startup = state["candidates_documents"][0]
        scores = state["evaluation_scores"]
        summary = state["evaluation_summary"]
        domain = state["domain"] or "기술"
        
        response = f"쿼리와 가장 유사한 스타트업을 찾았습니다:\n\n"
        response += f"이름: {startup['name']}\n"
        response += f"도메인: {domain}\n"
        response += f"설명: {startup['summary']}\n\n"
        
        if scores:
            response += "평가 점수:\n"
            response += f"- 팀 신뢰성: {scores['team_trustworthiness']}/10\n"
            response += f"- 고객 피드백: {scores['customer_feedback']}/10\n"
            response += f"- 리스크 분석: {scores['risk_analysis']}/10\n"
            response += f"- 창업자 헌신도: {scores['founder_commitment']}/10\n\n"
        
        if summary:
            response += f"평가 요약:\n{summary}"
    else:
        response = "관련 스타트업을 찾지 못했습니다."
    
    # 응답 메시지 추가
    state["messages"].append({"role": "assistant", "content": response})
    
    return state

# LangGraph 워크플로우 정의
def create_startup_evaluation_graph():
    """스타트업 평가 그래프 생성"""
    # 그래프 정의
    workflow = StateGraph(GraphState)
    
    # 노드 추가
    workflow.add_node("process_query", process_user_query)
    workflow.add_node("search_startups", search_startups)
    workflow.add_node("generate_summaries", generate_summaries)
    workflow.add_node("generate_evaluation_scores", generate_evaluation_scores)
    workflow.add_node("generate_evaluation_summary", generate_evaluation_summary)
    workflow.add_node("prepare_response", prepare_response)
    
    # 엣지 연결
    workflow.add_edge(START, "process_query")
    workflow.add_edge("process_query", "search_startups")
    workflow.add_edge("search_startups", "generate_summaries")
    workflow.add_edge("generate_summaries", "generate_evaluation_scores")
    workflow.add_edge("generate_evaluation_scores", "generate_evaluation_summary")
    workflow.add_edge("generate_evaluation_summary", "prepare_response")
    workflow.add_edge("prepare_response", END)
    
    # 그래프 컴파일
    return workflow.compile()

# 쿼리 처리 함수
def process_query(query: str, graph):
    """사용자 쿼리 처리 및 결과 반환"""
    # 초기 상태 설정
    initial_state: GraphState = {
        "messages": [{"role": "user", "content": query}],
        "user_query": "",
        "candidates_documents": [],
        "domain": None,
        "evaluation_scores": None,
        "evaluation_summary": None
    }
    
    # 그래프 실행
    result = graph.invoke(initial_state)
    
    # 결과 출력
    logger.info("\n--- 검색 결과 ---")
    logger.info(f"쿼리: {result['user_query']}")
    if result['domain']:
        logger.info(f"도메인: {result['domain']}")
    
    logger.info("\n최종 상태:")
    print_state(result)
    
    return result

# 상태 출력 함수
def print_state(state: GraphState):
    """상태를 보기 좋게 출력"""
    print("{\n  \"user_query\": \"" + state["user_query"] + "\",")
    
    # 후보 문서 출력
    print("  \"candidates_documents\": [")
    for i, doc in enumerate(state["candidates_documents"]):
        print("    {")
        print(f"      \"name\": \"{doc['name']}\",")
        # 요약이 여러 줄로 되어 있을 수 있으므로 줄바꿈 처리
        summary_lines = doc['summary'].split('\n')
        summary_formatted = '\\n'.join(summary_lines)
        print(f"      \"summary\": \"{summary_formatted}\"")
        print("    }" + ("," if i < len(state["candidates_documents"]) - 1 else ""))
    print("  ],")
    
    # 도메인 출력
    domain_value = f"\"{state['domain']}\"" if state['domain'] else "null"
    print(f"  \"domain\": {domain_value},")
    
    # 평가 점수 출력
    if state["evaluation_scores"]:
        print("  \"evaluation_scores\": {")
        scores = state["evaluation_scores"]
        print(f"    \"team_trustworthiness\": {scores['team_trustworthiness']},")
        print(f"    \"customer_feedback\": {scores['customer_feedback']},")
        print(f"    \"risk_analysis\": {scores['risk_analysis']},")
        print(f"    \"founder_commitment\": {scores['founder_commitment']}")
        print("  },")
    else:
        print("  \"evaluation_scores\": null,")
    
    # 평가 요약 출력
    if state["evaluation_summary"]:
        # 요약이 여러 줄로 되어 있을 수 있으므로 줄바꿈 처리
        summary_lines = state["evaluation_summary"].split('\n')
        summary_formatted = '\\n'.join(summary_lines)
        print(f"  \"evaluation_summary\": \"{summary_formatted}\"")
    else:
        print("  \"evaluation_summary\": null")
    
    print("}")

# 메인 함수
def main():
    # API 키 설정
    PINECONE_API_KEY = ""
    # OpenAI API 키는 환경 변수에서 가져오거나 직접 입력
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "")
    
    # Pinecone 인덱스 이름
    INDEX_NAME = "startup-index"
    
    # CSV 파일 경로
    CSV_PATH = r"C:\Users\lucy8\skala\startup_evaluation_agent\startup_data.csv"
    
    # 리소스 초기화
    init_resources(PINECONE_API_KEY, OPENAI_API_KEY, INDEX_NAME, CSV_PATH)
    
    # 그래프 생성
    graph = create_startup_evaluation_graph()
    
    # 사용자 쿼리 입력 받기
    user_query = input("관심 있는 스타트업 분야를 입력하세요: ")
    
    # 쿼리 처리 및 결과 출력
    result = process_query(user_query, graph)

if __name__ == "__main__":
    main()

2025-05-14 16:03:42,399 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-05-14 16:03:42,665 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device: cpu
2025-05-14 16:03:42,856 - __main__ - INFO - CSV 파일에서 5 개의 스타트업 데이터를 로드했습니다.
2025-05-14 16:03:42,857 - __main__ - INFO - 리소스 초기화 완료. 인덱스: startup-index


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-05-14 16:03:49,367 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-14 16:03:49,369 - __main__ - INFO - 스타트업 'addd'의 도메인으로 '소프트웨어'을(를) 추출했습니다.
2025-05-14 16:03:49,370 - __main__ - INFO - 쿼리와 가장 유사한 스타트업을 찾았습니다: addd
2025-05-14 16:03:51,081 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-14 16:03:51,087 - __main__ - INFO - 스타트업 설명 생성 완료: addd
2025-05-14 16:03:51,996 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-14 16:03:52,000 - __main__ - INFO - 평가 점수 응답: 팀 신뢰성: 8, 고객 피드백: 7, 리스크 분석: 6, 창업자 헌신도: 9
2025-05-14 16:03:52,001 - __main__ - INFO - 생성된 평가 점수: {'team_trustworthiness': 8, 'customer_feedback': 7, 'risk_analysis': 6, 'founder_commitment': 9}
2025-05-14 16:03:53,805 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-14 16:03:53,808 - __main__ - INFO - 생성된

{
  "user_query": "로봇",
  "candidates_documents": [
    {
      "name": "addd",
      "summary": "'addd'는 2019년에 설립된 한국계 스타트업으로, 광고마케팅 및 마케팅솔루션 분야에서 활동하고 있습니다. 주력 제품/서비스는 AI 기반 광고효과 측정 분석 및 모빌리티 광고매칭 플랫폼인 '달고'입니다. 또한, 스마트 모빌리티"
    }
  ],
  "domain": "소프트웨어",
  "evaluation_scores": {
    "team_trustworthiness": 8,
    "customer_feedback": 7,
    "risk_analysis": 6,
    "founder_commitment": 9
  },
  "evaluation_summary": "'addd'는 팀 신뢰성과 창업자 헌신도가 높아 투자가치가 있습니다. 그러나 고객 피드백과 리스크 분석이 부족하며 경쟁 치열한 광고마케팅 분야에서의 성공은 미지수입니다. 향후 고객 피드백과 리스크 관리에 더욱 집중하며, 시장에서의 명확한 경쟁우위를 구축해 나갈 필요가 있습니다."
}
